# NYC Open Data Design Document

## Overview
This document outlines the design and implementation strategy for fetching data from the NYC Open Data portal. The NYC Open Data portal provides access to hundreds of datasets from various city agencies through a comprehensive API system.

## Data Portal Information
- **Portal URL**: https://opendata.cityofnewyork.us/
- **Data Catalog**: https://data.cityofnewyork.us/browse
- **Technology**: Socrata Open Data API (SODA)
- **API Documentation**: https://dev.socrata.com/

## Key Features of NYC Open Data
- Over 3,000+ datasets from 80+ NYC agencies
- Real-time and historical data
- Multiple data formats (JSON, CSV, XML, RDF)
- Robust filtering and querying capabilities
- Geographic data support
- Open and free access (with optional registration for higher limits)

## API Structure and Endpoints

### SODA API (Socrata Open Data API)
NYC Open Data uses the Socrata platform, which provides a RESTful API called SODA (Socrata Open Data API).

#### Base URL Structure
```
https://data.cityofnewyork.us/resource/{dataset_id}.{format}
```

#### Supported Formats
- **JSON** (recommended): `.json`
- **CSV**: `.csv` 
- **XML**: `.xml`
- **RDF**: `.rdf`

#### Dataset Identification
Each dataset has a unique identifier (e.g., `erm2-nwe9` for 311 Service Requests). You can find this ID:
1. In the dataset URL
2. Via the "API" button on dataset pages
3. Through the Discovery API

### Authentication and Rate Limiting

#### Without App Token
- **Rate Limit**: Shared pool, limited requests
- **Throttling**: Aggressive throttling after modest usage
- **Use Case**: Testing and light development only

#### With App Token (Recommended)
- **Rate Limit**: 1,000 requests per rolling hour
- **Registration**: Free account required
- **Higher Limits**: Available by request for production apps

#### Getting an App Token
1. Create account at https://support.socrata.com/
2. Register for an app token
3. Include in requests as `X-App-Token` header or `$$app_token` parameter

## Data Fetching Architecture

### 1. Discovery and Dataset Selection
```
Discovery Phase
├── Browse catalog programmatically
├── Search datasets by keywords/categories
├── Identify relevant datasets
└── Extract dataset IDs and metadata
```

### 2. Data Access Layer
```
API Client Architecture
├── HTTP Client (requests/aiohttp)
├── Authentication Manager
├── Rate Limiter
├── Error Handler & Retry Logic
└── Response Parser
```

### 3. Query and Filtering Strategy
The SODA API supports SoQL (Socrata Query Language) for powerful data filtering:

#### Common Query Parameters
- `$select`: Choose specific columns
- `$where`: Filter rows (SQL-like conditions)
- `$order`: Sort results
- `$limit`: Number of records (max 50,000 per request)
- `$offset`: Pagination offset
- `$group`: Group by columns

#### Geographic Queries
- `within_circle(location, latitude, longitude, radius)`
- `within_box(location, north, east, south, west)`
- `intersects(location, 'MULTIPOLYGON(...)')`

### 4. Data Processing Pipeline
```
Processing Flow
├── Raw Data Ingestion
├── Data Validation & Cleaning
├── Schema Normalization
├── Duplicate Detection
├── Data Transformation
└── Storage/Output
```

### 5. Storage and Caching Strategy
- **Local Cache**: For frequently accessed datasets
- **Database Storage**: PostgreSQL/SQLite for structured data
- **File Storage**: Parquet/CSV for batch processing
- **Memory Management**: Chunked processing for large datasets

## Implementation Examples

### Required Dependencies

In [2]:
# Required libraries for NYC Open Data access
import requests
import pandas as pd
import json
from datetime import datetime, timedelta
import time
from typing import Dict, List, Optional
from urllib.parse import urlencode

In [3]:
class NYCOpenDataClient:
    """
    A client for accessing NYC Open Data via the SODA API
    """
    
    def __init__(self, app_token: Optional[str] = None):
        self.base_url = "https://data.cityofnewyork.us/resource"
        self.app_token = app_token
        self.session = requests.Session()
        
        # Set up headers
        if self.app_token:
            self.session.headers.update({
                'X-App-Token': self.app_token
            })
        
        # Rate limiting
        self.last_request_time = 0
        self.min_request_interval = 1.0  # 1 second between requests
    
    def _rate_limit(self):
        """Implement basic rate limiting"""
        current_time = time.time()
        time_since_last = current_time - self.last_request_time
        
        if time_since_last < self.min_request_interval:
            sleep_time = self.min_request_interval - time_since_last
            time.sleep(sleep_time)
        
        self.last_request_time = time.time()
    
    def get_dataset(self, dataset_id: str, **params) -> pd.DataFrame:
        """
        Fetch data from a specific dataset
        
        Args:
            dataset_id: The dataset identifier (e.g., 'erm2-nwe9')
            **params: SoQL query parameters
        
        Returns:
            pandas DataFrame with the results
        """
        self._rate_limit()
        
        url = f"{self.base_url}/{dataset_id}.json"
        
        try:
            response = self.session.get(url, params=params)
            response.raise_for_status()
            
            data = response.json()
            return pd.DataFrame(data)
            
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data: {e}")
            return pd.DataFrame()
    
    def search_datasets(self, query: str, limit: int = 10) -> List[Dict]:
        """
        Search for datasets using the Discovery API
        """
        # This would use the Discovery API endpoint
        # For now, returning a placeholder
        return []

In [4]:
# Example usage: Fetching 311 Service Requests

# Initialize client (replace with your app token for production)
client = NYCOpenDataClient(app_token=None)  # Use None for testing

# Example 1: Get recent 311 service requests
recent_311 = client.get_dataset(
    "erm2-nwe9",  # 311 Service Requests dataset ID
    **{
        "$limit": 1000,
        "$where": f"created_date >= '{(datetime.now() - timedelta(days=7)).isoformat()}'",
        "$order": "created_date DESC"
    }
)

print(f"Fetched {len(recent_311)} recent 311 service requests")
if not recent_311.empty:
    print("Columns:", list(recent_311.columns))
    print("\nFirst few rows:")
    print(recent_311.head())

Fetched 1000 recent 311 service requests
Columns: ['unique_key', 'created_date', 'agency', 'agency_name', 'complaint_type', 'descriptor', 'location_type', 'incident_zip', 'incident_address', 'street_name', 'cross_street_1', 'cross_street_2', 'intersection_street_1', 'intersection_street_2', 'address_type', 'city', 'landmark', 'status', 'community_board', 'bbl', 'borough', 'x_coordinate_state_plane', 'y_coordinate_state_plane', 'open_data_channel_type', 'park_facility_name', 'park_borough', 'latitude', 'longitude', 'location', 'resolution_action_updated_date', 'vehicle_type', 'closed_date', 'resolution_description', 'taxi_pick_up_location', 'facility_type', 'bridge_highway_name', 'road_ramp', 'bridge_highway_segment']

First few rows:
  unique_key             created_date agency                      agency_name  \
0   66119347  2025-09-12T01:50:45.000   NYPD  New York City Police Department   
1   66124986  2025-09-12T01:50:11.000   NYPD  New York City Police Department   
2   66118121 

In [5]:
# Example 2: Geographic filtering - Find issues in specific borough
manhattan_issues = client.get_dataset(
    "erm2-nwe9",
    **{
        "$limit": 500,
        "$where": "borough = 'MANHATTAN' AND created_date >= '2024-01-01'",
        "$select": "unique_key,complaint_type,descriptor,incident_address,borough,created_date,status"
    }
)

print(f"Found {len(manhattan_issues)} issues in Manhattan")

# Example 3: Aggregation query - Count by complaint type
complaint_counts = client.get_dataset(
    "erm2-nwe9", 
    **{
        "$select": "complaint_type,COUNT(*) as count",
        "$group": "complaint_type",
        "$order": "count DESC",
        "$limit": 20,
        "$where": "created_date >= '2024-01-01'"
    }
)

print("\nTop complaint types:")
print(complaint_counts)

Found 500 issues in Manhattan

Top complaint types:
              complaint_type   count
0            Illegal Parking  906103
1        Noise - Residential  666024
2             HEAT/HOT WATER  426799
3    Noise - Street/Sidewalk  292667
4           Blocked Driveway  285416
5       UNSANITARY CONDITION  173456
6               Water System  127722
7           Street Condition  126847
8          Abandoned Vehicle  119864
9         Noise - Commercial  113102
10           Dirty Condition  103309
11                  PLUMBING   99544
12                     Noise   93473
13             PAINT/PLASTER   92786
14         Derelict Vehicles   81058
15           Noise - Vehicle   78773
16  Traffic Signal Condition   78488
17                Encampment   76028
18               DOOR/WINDOW   68336
19           Illegal Dumping   67765

Top complaint types:
              complaint_type   count
0            Illegal Parking  906103
1        Noise - Residential  666024
2             HEAT/HOT WATER  426799
3

## Popular NYC Open Datasets

### High-Value Datasets for Analysis

1. **311 Service Requests** (`erm2-nwe9`)
   - Real-time citizen complaints and service requests
   - Geographic data, timestamps, complaint types
   - Updated daily, ~3M+ records

2. **NYPD Complaint Data** (`qgea-i56i`)
   - Historical crime complaint data
   - Geographic coordinates, offense types, demographics
   - Updated quarterly

3. **DOF Property Sales** (`34kk-4zzr`)
   - Real estate transaction data
   - Sale prices, property details, geographic data
   - Updated monthly

4. **Taxi & Limousine Commission Trip Data** (multiple datasets)
   - Yellow Taxi: `2yzn-sicd`, Green Taxi: `q5mz-t52e`
   - Trip details, pickup/dropoff locations, fares
   - Updated monthly, millions of trips

5. **NYC WiFi Hotspot Locations** (`yjub-udmw`)
   - Public WiFi access points across the city
   - Location data, provider information
   - Updated regularly

6. **Restaurant Inspection Results** (`43nn-pn8j`)
   - Health department inspections
   - Violation details, grades, restaurant info
   - Updated daily

### Data Quality Considerations
- **Missing Values**: Always check for null/empty fields
- **Data Types**: Dates and numbers may come as strings
- **Coordinate Systems**: Geographic data typically in WGS84
- **Update Frequency**: Varies by dataset (daily to quarterly)
- **Data Size**: Large datasets may require pagination

## Best Practices and Recommendations

### 1. Production Deployment
- **Always use App Tokens** for production applications
- **Implement robust error handling** for network issues
- **Cache frequently accessed data** to reduce API calls
- **Monitor rate limits** and implement exponential backoff
- **Use HTTPS** for all API requests

### 2. Performance Optimization
- **Selective Column Retrieval**: Use `$select` to limit columns
- **Efficient Filtering**: Apply `$where` clauses to reduce data transfer
- **Pagination Strategy**: Process large datasets in chunks
- **Concurrent Requests**: Use async programming for multiple datasets
- **Data Compression**: Request compressed responses when available

### 3. Data Processing Tips
- **Validate Data Types**: Convert strings to appropriate types
- **Handle Time Zones**: NYC data typically uses EST/EDT
- **Geographic Preprocessing**: Validate coordinate ranges
- **Duplicate Detection**: Use unique identifiers when available
- **Data Freshness**: Check last updated timestamps

### 4. Security and Compliance
- **Secure App Tokens**: Never expose tokens in public repositories
- **Respect Rate Limits**: Don't abuse the free service
- **Data Privacy**: Follow NYC's data use policies
- **Attribution**: Credit NYC Open Data in your applications

### 5. Monitoring and Logging
- **Request Logging**: Track API usage patterns
- **Error Monitoring**: Alert on failed requests
- **Performance Metrics**: Monitor response times
- **Data Quality Checks**: Validate expected data formats

## Next Steps
1. Register for an app token at the Socrata developer portal
2. Explore specific datasets relevant to your use case
3. Implement data validation and cleaning pipelines
4. Set up monitoring and alerting for production systems
5. Consider using specialized libraries like `sodapy` for Python

In [1]:
# Import Polars and other required libraries for motor vehicle accidents analysis
import polars as pl
import requests
import json
from datetime import datetime, timedelta
from typing import Dict, List, Optional

In [8]:
class NYCOpenDataPolarsClient:
    """
    NYC Open Data client optimized for Polars DataFrames
    """
    
    def __init__(self, app_token: Optional[str] = None):
        self.base_url = "https://data.cityofnewyork.us/resource"
        self.app_token = app_token
        self.session = requests.Session()
        
        if self.app_token:
            self.session.headers.update({'X-App-Token': self.app_token})
    
    def get_dataset_polars(self, dataset_id: str, **params) -> pl.DataFrame:
        """
        Fetch data and return as Polars DataFrame
        """
        url = f"{self.base_url}/{dataset_id}.json"
        
        try:
            response = self.session.get(url, params=params)
            response.raise_for_status()
            
            data = response.json()
            return pl.DataFrame(data)
            
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data: {e}")
            return pl.DataFrame()

# Initialize client
client = NYCOpenDataPolarsClient()

NameError: name 'pl' is not defined

In [6]:
# Fetch motor vehicle collision data
# Dataset ID: h9gi-nx95 (Motor Vehicle Collisions - Crashes)

print("Fetching motor vehicle collision data...")

# Get recent collision data (last 30 days)
recent_date = (datetime.now() - timedelta(days=30)).strftime('%Y-%m-%d')

collisions_df = client.get_dataset_polars(
    "h9gi-nx95",
    **{
        "$limit": 5000,
        "$where": f"crash_date >= '{recent_date}'",
        "$order": "crash_date DESC"
    }
)

print(f"Fetched {len(collisions_df)} motor vehicle collision records")
print(f"Dataset shape: {collisions_df.shape}")
print(f"Columns: {collisions_df.columns}")

Fetching motor vehicle collision data...


AttributeError: 'NYCOpenDataClient' object has no attribute 'get_dataset_polars'

In [4]:
# Explore the data structure and basic statistics
if not collisions_df.is_empty():
    print("=== DATASET OVERVIEW ===")
    print(f"Shape: {collisions_df.shape}")
    print(f"Memory usage: {collisions_df.estimated_size('mb'):.2f} MB")
    
    print("\n=== COLUMN INFORMATION ===")
    print(collisions_df.schema)
    
    print("\n=== FIRST FEW ROWS ===")
    print(collisions_df.head())
    
    print("\n=== DATA TYPES ===")
    for col, dtype in collisions_df.schema.items():
        print(f"{col}: {dtype}")
else:
    print("No data retrieved. Check your internet connection or try a different date range.")

=== DATASET OVERVIEW ===
Shape: (5000, 29)
Memory usage: 1.04 MB

=== COLUMN INFORMATION ===
Schema({'crash_date': String, 'crash_time': String, 'borough': String, 'zip_code': String, 'latitude': String, 'longitude': String, 'location': Struct({'latitude': String, 'longitude': String, 'human_address': String}), 'cross_street_name': String, 'number_of_persons_injured': String, 'number_of_persons_killed': String, 'number_of_pedestrians_injured': String, 'number_of_pedestrians_killed': String, 'number_of_cyclist_injured': String, 'number_of_cyclist_killed': String, 'number_of_motorist_injured': String, 'number_of_motorist_killed': String, 'contributing_factor_vehicle_1': String, 'contributing_factor_vehicle_2': String, 'collision_id': String, 'vehicle_type_code1': String, 'vehicle_type_code2': String, 'on_street_name': String, 'off_street_name': String, 'contributing_factor_vehicle_3': String, 'contributing_factor_vehicle_4': String, 'vehicle_type_code_3': String, 'vehicle_type_code_4': S

In [5]:
# Polars-powered analysis of motor vehicle collisions

if not collisions_df.is_empty():
    print("=== COLLISION ANALYSIS WITH POLARS ===")
    
    # 1. Count collisions by borough
    print("\n1. Collisions by Borough:")
    borough_counts = (collisions_df
                     .filter(pl.col("borough").is_not_null())
                     .group_by("borough")
                     .agg(pl.len().alias("collision_count"))
                     .sort("collision_count", descending=True))
    print(borough_counts)
    
    # 2. Count by contributing factors
    print("\n2. Top Contributing Factors:")
    contributing_factors = (collisions_df
                          .filter(pl.col("contributing_factor_vehicle_1").is_not_null())
                          .group_by("contributing_factor_vehicle_1")
                          .agg(pl.len().alias("count"))
                          .sort("count", descending=True)
                          .head(10))
    print(contributing_factors)
    
    # 3. Injury analysis
    print("\n3. Injury Statistics:")
    injury_stats = (collisions_df
                   .select([
                       pl.col("number_of_persons_injured").cast(pl.Int32, strict=False).sum().alias("total_injured"),
                       pl.col("number_of_persons_killed").cast(pl.Int32, strict=False).sum().alias("total_killed"),
                       pl.col("number_of_pedestrians_injured").cast(pl.Int32, strict=False).sum().alias("pedestrians_injured"),
                       pl.col("number_of_pedestrians_killed").cast(pl.Int32, strict=False).sum().alias("pedestrians_killed"),
                       pl.col("number_of_cyclist_injured").cast(pl.Int32, strict=False).sum().alias("cyclists_injured"),
                       pl.col("number_of_cyclist_killed").cast(pl.Int32, strict=False).sum().alias("cyclists_killed")
                   ]))
    print(injury_stats)

=== COLLISION ANALYSIS WITH POLARS ===

1. Collisions by Borough:
shape: (5, 2)
┌───────────────┬─────────────────┐
│ borough       ┆ collision_count │
│ ---           ┆ ---             │
│ str           ┆ u32             │
╞═══════════════╪═════════════════╡
│ BROOKLYN      ┆ 1459            │
│ QUEENS        ┆ 1061            │
│ MANHATTAN     ┆ 778             │
│ BRONX         ┆ 577             │
│ STATEN ISLAND ┆ 175             │
└───────────────┴─────────────────┘

2. Top Contributing Factors:
shape: (10, 2)
┌────────────────────────────────┬───────┐
│ contributing_factor_vehicle_1  ┆ count │
│ ---                            ┆ ---   │
│ str                            ┆ u32   │
╞════════════════════════════════╪═══════╡
│ Unspecified                    ┆ 1324  │
│ Driver Inattention/Distraction ┆ 1205  │
│ Failure to Yield Right-of-Way  ┆ 295   │
│ Following Too Closely          ┆ 280   │
│ Other Vehicular                ┆ 210   │
│ Passing or Lane Usage Improper ┆ 182   │
│ Unsa

In [8]:
# Advanced Polars analysis: Time-based patterns

if not collisions_df.is_empty():
    print("=== TIME-BASED ANALYSIS ===")
    
    # Convert crash_date to datetime if it exists
    if "crash_date" in collisions_df.columns:
        # Parse crash_date and crash_time
        df_with_datetime = (collisions_df
                           .with_columns([
                               pl.col("crash_date").str.strptime(pl.Date, "%Y-%m-%dT%H:%M:%S%.f").alias("date"),
                               pl.col("crash_time").alias("time_str")
                           ])
                           .filter(pl.col("date").is_not_null()))
        
        # 1. Collisions by day of week
        print("\n1. Collisions by Day of Week:")
        daily_pattern = (df_with_datetime
                        .with_columns(pl.col("date").dt.strftime("%A").alias("day_of_week"))
                        .group_by("day_of_week")
                        .agg(pl.len().alias("collision_count"))
                        .sort("collision_count", descending=True))
        print(daily_pattern)
        
        # 2. Most dangerous streets/locations
        print("\n2. Most Dangerous Streets:")
        dangerous_streets = (collisions_df
                           .filter(pl.col("on_street_name").is_not_null())
                           .group_by("on_street_name")
                           .agg([
                               pl.len().alias("collision_count"),
                               pl.col("number_of_persons_injured").cast(pl.Int32, strict=False).sum().alias("total_injured")
                           ])
                           .filter(pl.col("collision_count") >= 3)  # Only streets with 3+ collisions
                           .sort("collision_count", descending=True)
                           .head(10))
        print(dangerous_streets)
        
        # 3. Vehicle type analysis
        print("\n3. Vehicle Types Involved:")
        vehicle_types = (collisions_df
                        .filter(pl.col("vehicle_type_code1").is_not_null())
                        .group_by("vehicle_type_code1")
                        .agg(pl.len().alias("count"))
                        .sort("count", descending=True)
                        .head(10))
        print(vehicle_types)

=== TIME-BASED ANALYSIS ===

1. Collisions by Day of Week:
shape: (7, 2)
┌─────────────┬─────────────────┐
│ day_of_week ┆ collision_count │
│ ---         ┆ ---             │
│ str         ┆ u32             │
╞═════════════╪═════════════════╡
│ Saturday    ┆ 866             │
│ Friday      ┆ 855             │
│ Sunday      ┆ 704             │
│ Thursday    ┆ 700             │
│ Tuesday     ┆ 658             │
│ Monday      ┆ 646             │
│ Wednesday   ┆ 571             │
└─────────────┴─────────────────┘

2. Most Dangerous Streets:
shape: (10, 3)
┌────────────────────────────┬─────────────────┬───────────────┐
│ on_street_name             ┆ collision_count ┆ total_injured │
│ ---                        ┆ ---             ┆ ---           │
│ str                        ┆ u32             ┆ i32           │
╞════════════════════════════╪═════════════════╪═══════════════╡
│ BELT PARKWAY               ┆ 66              ┆ 45            │
│ LONG ISLAND EXPRESSWAY     ┆ 47              ┆ 50 

In [10]:
# Polars advanced filtering and aggregation examples

if not collisions_df.is_empty():
    print("=== ADVANCED POLARS OPERATIONS ===")
    
    # 1. Complex filtering: High-injury accidents
    print("\n1. High-Injury Accidents (3+ people injured):")
    high_injury_accidents = (collisions_df
                           .filter(pl.col("number_of_persons_injured").cast(pl.Int32, strict=False) >= 3)
                           .select([
                               "crash_date", "crash_time", "borough", "on_street_name", 
                               "number_of_persons_injured", "number_of_persons_killed",
                               "contributing_factor_vehicle_1"
                           ])
                           .head(10))
    print(high_injury_accidents)
    
    # 2. Window functions: Running totals
    print("\n2. Daily Collision Trends (Running Total):")
    if "crash_date" in collisions_df.columns:
        daily_trends = (collisions_df
                       .with_columns(pl.col("crash_date").str.strptime(pl.Date, "%Y-%m-%dT%H:%M:%S%.f").alias("date"))
                       .filter(pl.col("date").is_not_null())
                       .group_by("date")
                       .agg(pl.len().alias("daily_count"))
                       .sort("date")
                       .with_columns(pl.col("daily_count").cum_sum().alias("running_total"))
                       .tail(10))  # Last 10 days
        print(daily_trends)
    
    # 3. Conditional aggregations
    print("\n3. Severity Analysis by Borough:")
    severity_by_borough = (collisions_df
                          .group_by("borough")
                          .agg([
                              pl.len().alias("total_collisions"),
                              pl.col("number_of_persons_injured").cast(pl.Int32, strict=False).sum().alias("total_injured"),
                              pl.col("number_of_persons_killed").cast(pl.Int32, strict=False).sum().alias("total_killed"),
                              (pl.col("number_of_persons_injured").cast(pl.Int32, strict=False) > 0).sum().alias("injury_collisions"),
                              (pl.col("number_of_persons_killed").cast(pl.Int32, strict=False) > 0).sum().alias("fatal_collisions")
                          ])
                          .with_columns([
                              (pl.col("total_injured") / pl.col("total_collisions")).alias("injuries_per_collision"),
                              (pl.col("injury_collisions") / pl.col("total_collisions") * 100).alias("injury_rate_pct")
                          ])
                          .sort("total_collisions", descending=True))
    print(severity_by_borough)

=== ADVANCED POLARS OPERATIONS ===

1. High-Injury Accidents (3+ people injured):
shape: (10, 7)
┌──────────────┬────────────┬───────────┬──────────────┬──────────────┬──────────────┬─────────────┐
│ crash_date   ┆ crash_time ┆ borough   ┆ on_street_na ┆ number_of_pe ┆ number_of_pe ┆ contributin │
│ ---          ┆ ---        ┆ ---       ┆ me           ┆ rsons_injure ┆ rsons_killed ┆ g_factor_ve │
│ str          ┆ str        ┆ str       ┆ ---          ┆ d            ┆ ---          ┆ hicle_1     │
│              ┆            ┆           ┆ str          ┆ ---          ┆ str          ┆ ---         │
│              ┆            ┆           ┆              ┆ str          ┆              ┆ str         │
╞══════════════╪════════════╪═══════════╪══════════════╪══════════════╪══════════════╪═════════════╡
│ 2025-09-07T0 ┆ 4:45       ┆ QUEENS    ┆ MERRICK BLVD ┆ 3            ┆ 0            ┆ Traffic     │
│ 0:00:00.000  ┆            ┆           ┆              ┆              ┆              ┆ Control 

## Summary: NYC Motor Vehicle Accidents Analysis with Polars

### Key Insights from the Data
We successfully analyzed **5,000 recent motor vehicle collision records** from NYC Open Data using Polars. Here's what we discovered:

### Polars Features Demonstrated

1. **Data Ingestion**: Direct JSON → Polars DataFrame conversion
2. **Type Casting**: Safe conversion with `strict=False` for handling mixed data types
3. **Filtering**: Complex conditions with `.filter()` and null handling
4. **Grouping & Aggregation**: Multi-column aggregations with `.group_by()` and `.agg()`
5. **Date/Time Processing**: String parsing with `.str.strptime()` and date formatting
6. **Window Functions**: Cumulative sums with `.cum_sum()` for running totals
7. **Conditional Logic**: Boolean expressions for calculating injury rates
8. **Method Chaining**: Fluent API for complex data transformations
9. **Performance**: Fast processing of 5K+ records with minimal memory usage

### NYC Motor Vehicle Collision Dataset Features
- **Dataset ID**: `h9gi-nx95`
- **29 columns** including location, time, injuries, contributing factors
- **Real-time updates** with comprehensive incident details
- **Geographic data** with latitude/longitude coordinates
- **Detailed categorization** of vehicles, injuries, and causes

### Data Quality Observations
- Some missing values in borough and street name fields
- Mixed data types requiring careful casting
- Date formatting needs attention for proper parsing
- Rich categorical data for contributing factors and vehicle types

This analysis demonstrates Polars' efficiency for real-world data exploration tasks with NYC's comprehensive motor vehicle collision dataset.

## Motor Vehicle Collision Dataset: Complete Column Documentation

The NYC Motor Vehicle Collision dataset contains comprehensive information about every vehicle collision reported to the NYPD. Let's document all available columns and their purposes:

In [11]:
# Get comprehensive column information from the Motor Vehicle Collision dataset
print("=== NYC MOTOR VEHICLE COLLISION DATASET COLUMNS ===")
print(f"Dataset ID: h9gi-nx95")
print(f"Total Columns: {len(collisions_df.columns)}")
print(f"Total Records in Sample: {collisions_df.height}")
print()

# Display column names, data types, and sample values
print("COLUMN DETAILS:")
print("-" * 80)

for i, col_name in enumerate(collisions_df.columns, 1):
    # Get data type
    dtype = collisions_df[col_name].dtype
    
    # Get sample non-null values
    sample_values = (collisions_df
                    .filter(pl.col(col_name).is_not_null())
                    .select(col_name)
                    .head(3)
                    .to_series()
                    .to_list())
    
    # Count non-null values
    non_null_count = collisions_df.filter(pl.col(col_name).is_not_null()).height
    null_percentage = ((collisions_df.height - non_null_count) / collisions_df.height * 100)
    
    print(f"{i:2d}. {col_name}")
    print(f"    Type: {dtype}")
    print(f"    Non-null: {non_null_count}/{collisions_df.height} ({100-null_percentage:.1f}%)")
    
    if sample_values:
        sample_str = ", ".join([str(v)[:50] + "..." if len(str(v)) > 50 else str(v) for v in sample_values[:2]])
        print(f"    Sample: {sample_str}")
    else:
        print(f"    Sample: [All null values]")
    print()

=== NYC MOTOR VEHICLE COLLISION DATASET COLUMNS ===
Dataset ID: h9gi-nx95
Total Columns: 29
Total Records in Sample: 5000

COLUMN DETAILS:
--------------------------------------------------------------------------------
 1. crash_date
    Type: String
    Non-null: 5000/5000 (100.0%)
    Sample: 2025-09-08T00:00:00.000, 2025-09-08T00:00:00.000

 2. crash_time
    Type: String
    Non-null: 5000/5000 (100.0%)
    Sample: 0:31, 0:05

 3. borough
    Type: String
    Non-null: 4050/5000 (81.0%)
    Sample: QUEENS, QUEENS

 4. zip_code
    Type: String
    Non-null: 4050/5000 (81.0%)
    Sample: 11354, 11106

 5. latitude
    Type: String
    Non-null: 4962/5000 (99.2%)
    Sample: 40.757412, 40.704494

 6. longitude
    Type: String
    Non-null: 4962/5000 (99.2%)
    Sample: -73.83357, -73.81743

 7. location
    Type: Struct({'latitude': String, 'longitude': String, 'human_address': String})
    Non-null: 4962/5000 (99.2%)
    Sample: {'latitude': '40.757412', 'longitude': '-73.83357'..

### Column Descriptions and Business Context

Based on the NYC Open Data documentation, here's what each column represents:

#### **Temporal Information**
- **`crash_date`** - Date when the collision occurred (YYYY-MM-DD format)
- **`crash_time`** - Time when the collision occurred (HH:MM format)

#### **Location Information**
- **`borough`** - NYC borough where collision occurred (Manhattan, Brooklyn, Queens, Bronx, Staten Island)
- **`zip_code`** - ZIP code of the collision location
- **`latitude`** - Geographic latitude coordinate
- **`longitude`** - Geographic longitude coordinate  
- **`location`** - Combined lat/long in POINT format for mapping
- **`on_street_name`** - Primary street where collision occurred
- **`off_street_name`** - Secondary/intersecting street
- **`cross_street_name`** - Cross street at intersection

#### **Injury Statistics**
- **`number_of_persons_injured`** - Total people injured in the collision
- **`number_of_persons_killed`** - Total people killed in the collision
- **`number_of_pedestrians_injured`** - Pedestrians injured
- **`number_of_pedestrians_killed`** - Pedestrians killed
- **`number_of_cyclist_injured`** - Cyclists injured
- **`number_of_cyclist_killed`** - Cyclists killed
- **`number_of_motorist_injured`** - Vehicle occupants injured
- **`number_of_motorist_killed`** - Vehicle occupants killed

#### **Contributing Factors**
- **`contributing_factor_vehicle_1`** - Primary cause for vehicle 1
- **`contributing_factor_vehicle_2`** - Primary cause for vehicle 2  
- **`contributing_factor_vehicle_3`** - Primary cause for vehicle 3
- **`contributing_factor_vehicle_4`** - Primary cause for vehicle 4
- **`contributing_factor_vehicle_5`** - Primary cause for vehicle 5

#### **Vehicle Information**
- **`vehicle_type_code1`** - Type of vehicle 1 (sedan, SUV, truck, etc.)
- **`vehicle_type_code2`** - Type of vehicle 2
- **`vehicle_type_code3`** - Type of vehicle 3
- **`vehicle_type_code4`** - Type of vehicle 4
- **`vehicle_type_code5`** - Type of vehicle 5

#### **Administrative Data**
- **`collision_id`** - Unique identifier for each collision report
- **`unique_key`** - Alternative unique identifier (often same as collision_id)

### Data Quality Notes
- **Missing Values**: Location fields (borough, street names) often have missing values
- **Contributing Factors**: May include values like "Unspecified", "Driver Inattention/Distraction", "Following Too Closely"
- **Vehicle Types**: Includes passenger vehicles, commercial vehicles, bicycles, motorcycles, etc.
- **Geographic Coverage**: All five NYC boroughs with precise lat/long coordinates when available

In [7]:
# Practical examples of working with key columns
print("=== PRACTICAL COLUMN USAGE EXAMPLES ===")
print()

# Example 1: Analyzing Contributing Factors
print("1. TOP CONTRIBUTING FACTORS:")
contributing_factors = (collisions_df
                       .select("contributing_factor_vehicle_1")
                       .filter(pl.col("contributing_factor_vehicle_1").is_not_null())
                       .group_by("contributing_factor_vehicle_1")
                       .agg(pl.count().alias("count"))
                       .sort("count", descending=True)
                       .head(5))
print(contributing_factors)
print()

# Example 2: Vehicle Type Analysis
print("2. MOST COMMON VEHICLE TYPES:")
vehicle_types = (collisions_df
                .select("vehicle_type_code1")
                .filter(pl.col("vehicle_type_code1").is_not_null())
                .group_by("vehicle_type_code1") 
                .agg(pl.count().alias("count"))
                .sort("count", descending=True)
                .head(5))
print(vehicle_types)
print()

# Example 3: Injury Severity Analysis
print("3. INJURY SEVERITY DISTRIBUTION:")
injury_analysis = (collisions_df
                  .select([
                      "number_of_persons_injured",
                      "number_of_persons_killed"
                  ])
                  .with_columns([
                      pl.col("number_of_persons_injured").cast(pl.Int32, strict=False),
                      pl.col("number_of_persons_killed").cast(pl.Int32, strict=False)
                  ])
                  .with_columns([
                      (pl.col("number_of_persons_injured") + pl.col("number_of_persons_killed")).alias("total_casualties")
                  ])
                  .group_by("total_casualties")
                  .agg(pl.count().alias("collision_count"))
                  .sort("total_casualties")
                  .head(8))
print(injury_analysis)
print()

# Example 4: Geographic Distribution
print("4. COLLISIONS BY BOROUGH:")
borough_summary = (collisions_df
                  .filter(pl.col("borough").is_not_null())
                  .group_by("borough")
                  .agg([
                      pl.count().alias("total_collisions"),
                      pl.col("number_of_persons_injured").cast(pl.Int32, strict=False).sum().alias("total_injured"),
                      pl.col("number_of_persons_killed").cast(pl.Int32, strict=False).sum().alias("total_killed")
                  ])
                  .sort("total_collisions", descending=True))
print(borough_summary)
print()

print("=== COLUMN USAGE TIPS ===")
print("• Use .cast(pl.Int32, strict=False) for numeric columns with missing values")
print("• Filter .is_not_null() before grouping to avoid null groups")
print("• Vehicle and factor columns often contain 'Unspecified' values")
print("• Lat/Long coordinates can be used for geographic analysis and mapping")
print("• Date/time columns enable temporal pattern analysis")

=== PRACTICAL COLUMN USAGE EXAMPLES ===

1. TOP CONTRIBUTING FACTORS:


NameError: name 'collisions_df' is not defined

## Geographic Visualization: Motor Vehicle Collisions on OpenStreetMap

Now let's create an interactive map visualization of the collision locations using the latitude and longitude coordinates from our dataset. We'll use Folium to overlay the collision data on OpenStreetMap.

In [4]:
# Install required mapping libraries
import subprocess
import sys

def install_package(package):
    """Install a package using pip"""
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✅ Successfully installed {package}")
    except subprocess.CalledProcessError as e:
        print(f"❌ Failed to install {package}: {e}")

# Install Folium for interactive maps
print("Installing mapping libraries...")
install_package("folium")

# Import mapping libraries
import folium
from folium import plugins
print("✅ Mapping libraries imported successfully!")
print(f"Folium version: {folium.__version__}")

Installing mapping libraries...
✅ Successfully installed folium
✅ Mapping libraries imported successfully!
Folium version: 0.20.0


In [5]:
# Prepare location data for mapping
print("=== PREPARING LOCATION DATA FOR MAPPING ===")

# Extract location data with valid coordinates
location_data = (collisions_df
                .select([
                    "collision_id",
                    "latitude", 
                    "longitude",
                    "borough",
                    "on_street_name",
                    "cross_street_name",
                    "crash_date",
                    "crash_time",
                    "number_of_persons_injured",
                    "number_of_persons_killed",
                    "contributing_factor_vehicle_1"
                ])
                .filter(
                    pl.col("latitude").is_not_null() & 
                    pl.col("longitude").is_not_null() &
                    (pl.col("latitude") != "0") &
                    (pl.col("longitude") != "0")
                )
                .with_columns([
                    pl.col("latitude").cast(pl.Float64, strict=False),
                    pl.col("longitude").cast(pl.Float64, strict=False),
                    pl.col("number_of_persons_injured").cast(pl.Int32, strict=False).fill_null(0),
                    pl.col("number_of_persons_killed").cast(pl.Int32, strict=False).fill_null(0)
                ])
                .filter(
                    # Valid NYC coordinates roughly
                    (pl.col("latitude") > 40.4) & 
                    (pl.col("latitude") < 40.9) &
                    (pl.col("longitude") > -74.3) & 
                    (pl.col("longitude") < -73.7)
                ))

print(f"Total collisions with valid coordinates: {location_data.height}")
print(f"Original dataset size: {collisions_df.height}")
print(f"Percentage with valid coordinates: {(location_data.height/collisions_df.height)*100:.1f}%")

# Show sample of location data
print("\nSample location data:")
print(location_data.head())

=== PREPARING LOCATION DATA FOR MAPPING ===
Total collisions with valid coordinates: 4854
Original dataset size: 5000
Percentage with valid coordinates: 97.1%

Sample location data:
shape: (5, 11)
┌────────────┬───────────┬───────────┬─────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ collision_ ┆ latitude  ┆ longitude ┆ borough ┆ … ┆ crash_tim ┆ number_of ┆ number_of ┆ contribut │
│ id         ┆ ---       ┆ ---       ┆ ---     ┆   ┆ e         ┆ _persons_ ┆ _persons_ ┆ ing_facto │
│ ---        ┆ f64       ┆ f64       ┆ str     ┆   ┆ ---       ┆ injured   ┆ killed    ┆ r_vehicle │
│ str        ┆           ┆           ┆         ┆   ┆ str       ┆ ---       ┆ ---       ┆ _1        │
│            ┆           ┆           ┆         ┆   ┆           ┆ i32       ┆ i32       ┆ ---       │
│            ┆           ┆           ┆         ┆   ┆           ┆           ┆           ┆ str       │
╞════════════╪═══════════╪═══════════╪═════════╪═══╪═══════════╪═══════════╪═══════════╪════════

In [6]:
# Create interactive map of motor vehicle collisions
print("=== CREATING INTERACTIVE COLLISION MAP ===")

# Calculate map center (NYC center)
if location_data.height > 0:
    center_lat = location_data.select(pl.col("latitude").mean()).item()
    center_lon = location_data.select(pl.col("longitude").mean()).item()
    
    print(f"Map center: {center_lat:.4f}, {center_lon:.4f}")
    
    # Create base map centered on NYC
    collision_map = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=11,
        tiles='OpenStreetMap'
    )
    
    # Convert to pandas for easier iteration (small dataset)
    location_pd = location_data.to_pandas()
    
    # Add collision points to map (limit to first 500 for performance)
    sample_size = min(500, len(location_pd))
    location_sample = location_pd.head(sample_size)
    
    print(f"Adding {len(location_sample)} collision points to map...")
    
    # Color coding based on severity
    def get_color_and_size(injured, killed):
        total_casualties = injured + killed
        if killed > 0:
            return 'red', 8  # Fatal accidents
        elif injured > 2:
            return 'orange', 6  # Multiple injuries
        elif injured > 0:
            return 'yellow', 4  # Minor injuries
        else:
            return 'blue', 3  # Property damage only
    
    # Add markers for each collision
    for idx, row in location_sample.iterrows():
        color, size = get_color_and_size(row['number_of_persons_injured'], row['number_of_persons_killed'])
        
        # Create popup text
        popup_text = f"""
        <b>Collision Details</b><br>
        <b>Date:</b> {row['crash_date']} {row['crash_time']}<br>
        <b>Location:</b> {row['on_street_name'] or 'Unknown'}<br>
        <b>Borough:</b> {row['borough'] or 'Unknown'}<br>
        <b>Injured:</b> {row['number_of_persons_injured']}<br>
        <b>Killed:</b> {row['number_of_persons_killed']}<br>
        <b>Contributing Factor:</b> {row['contributing_factor_vehicle_1'] or 'Unknown'}
        """
        
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=size,
            popup=popup_text,
            color='black',
            weight=1,
            fillColor=color,
            fillOpacity=0.7
        ).add_to(collision_map)
    
    # Add a legend
    legend_html = '''
    <div style="position: fixed; 
                bottom: 50px; left: 50px; width: 200px; height: 120px; 
                background-color: white; border:2px solid grey; z-index:9999; 
                font-size:14px; padding: 10px">
    <p><b>Collision Severity</b></p>
    <p><i class="fa fa-circle" style="color:red"></i> Fatal (killed > 0)</p>
    <p><i class="fa fa-circle" style="color:orange"></i> Multiple Injuries (>2)</p>
    <p><i class="fa fa-circle" style="color:yellow"></i> Minor Injuries (1-2)</p>
    <p><i class="fa fa-circle" style="color:blue"></i> Property Damage Only</p>
    </div>
    '''
    collision_map.get_root().html.add_child(folium.Element(legend_html))
    
    # Save the map
    map_filename = "nyc_motor_vehicle_collisions_map.html"
    collision_map.save(map_filename)
    
    print(f"✅ Map saved as: {map_filename}")
    print(f"📊 Map contains {len(location_sample)} collision points")
    print(f"🗺️ Interactive map with OpenStreetMap tiles")
    
    # Display map in notebook
    collision_map
    
else:
    print("❌ No valid location data found for mapping")

=== CREATING INTERACTIVE COLLISION MAP ===
Map center: 40.7189, -73.9187
Adding 500 collision points to map...
✅ Map saved as: nyc_motor_vehicle_collisions_map.html
📊 Map contains 500 collision points
🗺️ Interactive map with OpenStreetMap tiles


In [7]:
# Create collision density heatmap
print("=== CREATING COLLISION DENSITY HEATMAP ===")

if location_data.height > 0:
    # Create heatmap
    heatmap = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=11,
        tiles='OpenStreetMap'
    )
    
    # Prepare heatmap data (lat, lon pairs)
    heat_data = [[row['latitude'], row['longitude']] for idx, row in location_sample.iterrows()]
    
    # Add heatmap layer
    plugins.HeatMap(heat_data, 
                   radius=15,
                   blur=10,
                   max_zoom=18).add_to(heatmap)
    
    # Save heatmap
    heatmap_filename = "nyc_collision_density_heatmap.html"
    heatmap.save(heatmap_filename)
    
    print(f"✅ Heatmap saved as: {heatmap_filename}")
    print(f"🔥 Shows collision density hotspots across NYC")
    
    # Display heatmap
    heatmap
    
else:
    print("❌ No valid location data found for heatmap")

=== CREATING COLLISION DENSITY HEATMAP ===
✅ Heatmap saved as: nyc_collision_density_heatmap.html
🔥 Shows collision density hotspots across NYC


In [8]:
# Geographic analysis insights
print("=== GEOGRAPHIC ANALYSIS INSIGHTS ===")

if location_data.height > 0:
    # Borough distribution of mapped collisions
    borough_map_stats = (location_data
                        .filter(pl.col("borough").is_not_null())
                        .group_by("borough")
                        .agg([
                            pl.len().alias("mapped_collisions"),
                            pl.col("number_of_persons_injured").sum().alias("total_injured"),
                            pl.col("number_of_persons_killed").sum().alias("total_killed")
                        ])
                        .with_columns([
                            (pl.col("total_injured") + pl.col("total_killed")).alias("total_casualties")
                        ])
                        .sort("mapped_collisions", descending=True))
    
    print("COLLISION DISTRIBUTION BY BOROUGH:")
    print(borough_map_stats)
    print()
    
    # Coordinate bounds
    lat_bounds = location_data.select([
        pl.col("latitude").min().alias("min_lat"),
        pl.col("latitude").max().alias("max_lat")
    ]).to_pandas().iloc[0]
    
    lon_bounds = location_data.select([
        pl.col("longitude").min().alias("min_lon"),
        pl.col("longitude").max().alias("max_lon")
    ]).to_pandas().iloc[0]
    
    print("GEOGRAPHIC BOUNDS OF MAPPED COLLISIONS:")
    print(f"Latitude: {lat_bounds['min_lat']:.4f} to {lat_bounds['max_lat']:.4f}")
    print(f"Longitude: {lon_bounds['min_lon']:.4f} to {lon_bounds['max_lon']:.4f}")
    print()
    
    # Time-based insights
    hourly_pattern = (location_data
                     .filter(pl.col("crash_time").is_not_null())
                     .with_columns([
                         pl.col("crash_time").str.slice(0, 2).cast(pl.Int32, strict=False).alias("hour")
                     ])
                     .filter(pl.col("hour").is_not_null())
                     .group_by("hour")
                     .agg(pl.len().alias("collision_count"))
                     .sort("hour"))
    
    print("COLLISION PATTERN BY HOUR OF DAY:")
    print(hourly_pattern.head(10))
    
print("""
=== MAPPING FEATURES CREATED ===
✅ Interactive Point Map: Each collision as a colored marker
   • Red: Fatal accidents (deaths > 0)
   • Orange: Multiple injuries (>2 injured)  
   • Yellow: Minor injuries (1-2 injured)
   • Blue: Property damage only
   
✅ Density Heatmap: Shows collision hotspots
   • Identifies high-risk areas
   • Useful for traffic safety planning
   
✅ Interactive Features:
   • Click markers for collision details
   • Zoom and pan functionality
   • OpenStreetMap base layer
   • Legend for severity coding

📁 Files Created:
   • nyc_motor_vehicle_collisions_map.html
   • nyc_collision_density_heatmap.html
""")

=== GEOGRAPHIC ANALYSIS INSIGHTS ===
COLLISION DISTRIBUTION BY BOROUGH:
shape: (5, 5)
┌───────────────┬───────────────────┬───────────────┬──────────────┬──────────────────┐
│ borough       ┆ mapped_collisions ┆ total_injured ┆ total_killed ┆ total_casualties │
│ ---           ┆ ---               ┆ ---           ┆ ---          ┆ ---              │
│ str           ┆ u32               ┆ i32           ┆ i32          ┆ i32              │
╞═══════════════╪═══════════════════╪═══════════════╪══════════════╪══════════════════╡
│ BROOKLYN      ┆ 1422              ┆ 904           ┆ 4            ┆ 908              │
│ QUEENS        ┆ 1034              ┆ 615           ┆ 2            ┆ 617              │
│ MANHATTAN     ┆ 746               ┆ 411           ┆ 1            ┆ 412              │
│ BRONX         ┆ 562               ┆ 365           ┆ 2            ┆ 367              │
│ STATEN ISLAND ┆ 171               ┆ 88            ┆ 0            ┆ 88               │
└───────────────┴─────────────────

## Spatial Clustering Analysis: Accident Hotspots by Borough

Let's perform spatial clustering analysis to identify accident hotspots within each NYC borough. We'll use K-means clustering to group nearby collision locations and analyze patterns.

In [10]:
# Install and import clustering libraries
print("Installing clustering libraries...")
install_package("scikit-learn")

# Import clustering libraries
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

print("✅ Clustering libraries imported successfully!")
print("📊 Ready for spatial clustering analysis")

Installing clustering libraries...
✅ Successfully installed scikit-learn
✅ Clustering libraries imported successfully!
📊 Ready for spatial clustering analysis


In [12]:
# Perform clustering analysis by borough
print("=== SPATIAL CLUSTERING ANALYSIS BY BOROUGH ===")

def cluster_accidents_by_borough(data, borough_name, n_clusters=5):
    """
    Perform K-means clustering on accident locations for a specific borough
    """
    # Filter data for specific borough
    borough_data = data.filter(pl.col("borough") == borough_name)
    
    if borough_data.height < n_clusters:
        print(f"⚠️ {borough_name}: Not enough data points ({borough_data.height}) for {n_clusters} clusters")
        return None, None
    
    # Extract coordinates
    coords = borough_data.select(["latitude", "longitude"]).to_pandas()
    
    if len(coords) == 0:
        return None, None
    
    # Standardize coordinates for clustering
    scaler = StandardScaler()
    coords_scaled = scaler.fit_transform(coords)
    
    # Perform K-means clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(coords_scaled)
    
    # Add cluster labels to original data
    borough_data_pd = borough_data.to_pandas()
    borough_data_pd['cluster'] = cluster_labels
    
    # Get cluster centers in original scale
    cluster_centers_scaled = kmeans.cluster_centers_
    cluster_centers = scaler.inverse_transform(cluster_centers_scaled)
    
    return borough_data_pd, cluster_centers

# Analyze each borough
borough_clusters = {}
cluster_centers_all = {}

# Get list of boroughs with valid data
valid_boroughs = (location_data
                 .filter(pl.col("borough").is_not_null())
                 .group_by("borough")
                 .agg(pl.len().alias("count"))
                 .filter(pl.col("count") >= 10)  # Minimum 10 accidents for clustering
                 .select("borough")
                 .to_series()
                 .to_list())

print(f"Analyzing {len(valid_boroughs)} boroughs with sufficient data...")
print(f"Boroughs: {valid_boroughs}")
print()

for borough in valid_boroughs:
    print(f"🏘️ Analyzing {borough}...")
    
    # Determine optimal number of clusters based on data size
    borough_size = location_data.filter(pl.col("borough") == borough).height
    n_clusters = min(5, max(2, borough_size // 20))  # 1 cluster per 20 accidents, max 5
    
    clustered_data, centers = cluster_accidents_by_borough(location_data, borough, n_clusters)
    
    if clustered_data is not None:
        borough_clusters[borough] = clustered_data
        cluster_centers_all[borough] = centers
        
        # Display cluster statistics
        cluster_stats = clustered_data.groupby('cluster').agg({
            'collision_id': 'count',
            'number_of_persons_injured': 'sum',
            'number_of_persons_killed': 'sum',
            'latitude': 'mean',
            'longitude': 'mean'
        }).round(4)
        
        print(f"   📍 {n_clusters} clusters created")
        print(f"   📊 {len(clustered_data)} accidents clustered")
        print("   Cluster Statistics:")
        print(cluster_stats)
        print()
    else:
        print(f"   ❌ Failed to cluster {borough}")
        print()

print(f"✅ Clustering completed for {len(borough_clusters)} boroughs")

=== SPATIAL CLUSTERING ANALYSIS BY BOROUGH ===
Analyzing 5 boroughs with sufficient data...
Boroughs: ['MANHATTAN', 'QUEENS', 'STATEN ISLAND', 'BRONX', 'BROOKLYN']

🏘️ Analyzing MANHATTAN...
   📍 5 clusters created
   📊 746 accidents clustered
   Cluster Statistics:
         collision_id  number_of_persons_injured  number_of_persons_killed  \
cluster                                                                      
0                 174                         89                         0   
1                 113                         70                         1   
2                  58                         25                         0   
3                 199                        105                         0   
4                 202                        121                         0   

         latitude  longitude  
cluster                       
0         40.7220   -73.9992  
1         40.7755   -73.9665  
2         40.8531   -73.9310  
3         40.8086   -73.9457  


In [8]:
# Create clustered maps for each borough
print("=== CREATING CLUSTERED MAPS BY BOROUGH ===")

# Define colors for clusters
cluster_colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 
                 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white', 
                 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']

def create_borough_cluster_map(borough_name, clustered_data, cluster_centers):
    """Create a map showing clusters for a specific borough"""
    
    if clustered_data is None or len(clustered_data) == 0:
        return None
    
    # Calculate map center
    center_lat = clustered_data['latitude'].mean()
    center_lon = clustered_data['longitude'].mean()
    
    # Create map
    cluster_map = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=12,
        tiles='OpenStreetMap'
    )
    
    # Add clustered points
    for idx, row in clustered_data.iterrows():
        cluster_id = int(row['cluster'])
        color = cluster_colors[cluster_id % len(cluster_colors)]
        
        # Create popup text
        popup_text = f"""
        <b>{borough_name} - Cluster {cluster_id}</b><br>
        <b>Date:</b> {row['crash_date']} {row['crash_time']}<br>
        <b>Street:</b> {row['on_street_name'] or 'Unknown'}<br>
        <b>Injured:</b> {row['number_of_persons_injured']}<br>
        <b>Killed:</b> {row['number_of_persons_killed']}<br>
        <b>Factor:</b> {row['contributing_factor_vehicle_1'] or 'Unknown'}
        """
        
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=5,
            popup=popup_text,
            color='black',
            weight=1,
            fillColor=color,
            fillOpacity=0.7
        ).add_to(cluster_map)
    
    # Add cluster centers as larger markers
    for i, center in enumerate(cluster_centers):
        color = cluster_colors[i % len(cluster_colors)]
        
        folium.Marker(
            location=[center[0], center[1]],
            popup=f"<b>Cluster {i} Center</b><br>{borough_name}",
            icon=folium.Icon(color='white', icon_color=color, icon='star')
        ).add_to(cluster_map)
    
    return cluster_map

# Create and save maps for each borough
cluster_maps = {}
for borough in borough_clusters.keys():
    print(f"📍 Creating cluster map for {borough}...")
    
    cluster_map = create_borough_cluster_map(
        borough, 
        borough_clusters[borough], 
        cluster_centers_all[borough]
    )
    
    if cluster_map:
        cluster_maps[borough] = cluster_map
        
        # Save map
        filename = f"clusters_{borough.lower().replace(' ', '_')}_collisions.html"
        cluster_map.save(filename)
        print(f"   ✅ Saved: {filename}")
    else:
        print(f"   ❌ Failed to create map for {borough}")

print(f"\n🗺️ Created {len(cluster_maps)} borough cluster maps")

=== CREATING CLUSTERED MAPS BY BOROUGH ===
📍 Creating cluster map for QUEENS...
   ✅ Saved: clusters_queens_collisions.html
📍 Creating cluster map for BROOKLYN...
   ✅ Saved: clusters_brooklyn_collisions.html
📍 Creating cluster map for MANHATTAN...
   ✅ Saved: clusters_manhattan_collisions.html
📍 Creating cluster map for BRONX...
   ✅ Saved: clusters_bronx_collisions.html
📍 Creating cluster map for STATEN ISLAND...
   ✅ Saved: clusters_staten_island_collisions.html

🗺️ Created 5 borough cluster maps


In [14]:
# Comprehensive cluster analysis and insights
print("=== COMPREHENSIVE CLUSTER ANALYSIS ===")

# Analyze cluster characteristics across all boroughs
all_cluster_stats = []

for borough, clustered_data in borough_clusters.items():
    # Reset index to ensure 'cluster' is a column
    clustered_data = clustered_data.reset_index(drop=True)
    
    cluster_summary = clustered_data.groupby('cluster').agg({
        'collision_id': 'count',
        'number_of_persons_injured': 'sum',
        'number_of_persons_killed': 'sum',
        'latitude': 'mean',
        'longitude': 'mean'
    }).reset_index()  # Make sure cluster becomes a column
    
    cluster_summary['borough'] = borough
    cluster_summary['total_casualties'] = (cluster_summary['number_of_persons_injured'] + 
                                          cluster_summary['number_of_persons_killed'])
    cluster_summary['accidents_per_cluster'] = cluster_summary['collision_id']
    cluster_summary['severity_rate'] = (cluster_summary['total_casualties'] / 
                                       cluster_summary['collision_id']).round(3)
    
    all_cluster_stats.append(cluster_summary)

if all_cluster_stats:
    # Combine all cluster statistics
    combined_stats = pd.concat(all_cluster_stats, ignore_index=True)
    
    print("🏆 TOP 10 MOST DANGEROUS CLUSTERS (by total casualties):")
    print("Available columns:", combined_stats.columns.tolist())
    
    if 'total_casualties' in combined_stats.columns:
        top_dangerous = combined_stats.nlargest(10, 'total_casualties')[
            ['borough', 'cluster', 'collision_id', 'total_casualties', 'severity_rate', 'latitude', 'longitude']
        ]
        print(top_dangerous)
        print()
        
        print("📊 TOP 10 HIGHEST ACCIDENT VOLUME CLUSTERS:")
        top_volume = combined_stats.nlargest(10, 'collision_id')[
            ['borough', 'cluster', 'collision_id', 'total_casualties', 'severity_rate', 'latitude', 'longitude']
        ]
        print(top_volume)
        print()
        
        # Borough-level cluster summary
        print("🏘️ BOROUGH CLUSTER SUMMARY:")
        borough_summary = combined_stats.groupby('borough').agg({
            'collision_id': 'sum',
            'total_casualties': 'sum',
            'cluster': 'count'
        }).round(2)
        borough_summary.columns = ['Total_Accidents', 'Total_Casualties', 'Num_Clusters']
        borough_summary['Avg_Accidents_Per_Cluster'] = (borough_summary['Total_Accidents'] / 
                                                       borough_summary['Num_Clusters']).round(1)
        borough_summary['Casualty_Rate'] = (borough_summary['Total_Casualties'] / 
                                           borough_summary['Total_Accidents']).round(3)
        print(borough_summary)
        print()
        
        # Find the most severe cluster in each borough
        print("⚠️ MOST SEVERE CLUSTER BY BOROUGH (highest casualty rate):")
        most_severe_by_borough = combined_stats.loc[combined_stats.groupby('borough')['severity_rate'].idxmax()][
            ['borough', 'cluster', 'collision_id', 'total_casualties', 'severity_rate', 'latitude', 'longitude']
        ]
        print(most_severe_by_borough)
        
        # Cluster distribution analysis
        print("\n📈 CLUSTER SIZE DISTRIBUTION:")
        cluster_sizes = combined_stats['collision_id']
        print(f"Mean accidents per cluster: {cluster_sizes.mean():.1f}")
        print(f"Median accidents per cluster: {cluster_sizes.median():.1f}")
        print(f"Largest cluster: {cluster_sizes.max()} accidents")
        print(f"Smallest cluster: {cluster_sizes.min()} accidents")
        print(f"Standard deviation: {cluster_sizes.std():.1f}")
    else:
        print("Error: Could not find expected columns in cluster data")
        print("Available columns:", combined_stats.columns.tolist())

print(f"""
=== CLUSTERING ANALYSIS SUMMARY ===
✅ Spatial Analysis Complete:
   • {len(borough_clusters)} boroughs analyzed
   • K-means clustering applied per borough
   • Cluster centers identified as accident hotspots
   • Color-coded maps created for each borough

📊 Key Insights:
   • Each cluster represents a geographic hotspot of accidents
   • Cluster centers show the most concentrated accident areas
   • Different colors represent different clusters within each borough
   • Star markers indicate cluster centers (hotspot epicenters)

🗺️ Files Created:""")

for borough in cluster_maps.keys():
    filename = f"clusters_{borough.lower().replace(' ', '_')}_collisions.html"
    print(f"   • {filename}")

print("""
💡 Use Cases:
   • Traffic safety planning and resource allocation
   • Identifying intersection improvements needed
   • Emergency response station positioning
   • Traffic enforcement priority areas
""")

=== COMPREHENSIVE CLUSTER ANALYSIS ===
🏆 TOP 10 MOST DANGEROUS CLUSTERS (by total casualties):
Available columns: ['cluster', 'collision_id', 'number_of_persons_injured', 'number_of_persons_killed', 'latitude', 'longitude', 'borough', 'total_casualties', 'accidents_per_cluster', 'severity_rate']
      borough  cluster  collision_id  total_casualties  severity_rate  \
22   BROOKLYN        2           419               299          0.714   
6      QUEENS        1           310               193          0.623   
5      QUEENS        0           309               188          0.608   
21   BROOKLYN        1           243               181          0.745   
24   BROOKLYN        4           271               153          0.565   
23   BROOKLYN        3           206               142          0.689   
20   BROOKLYN        0           289               135          0.467   
8      QUEENS        3           231               124          0.537   
4   MANHATTAN        4           202          

In [1]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

class PlotlyMappingClient:
    """
    A comprehensive abstraction class for creating interactive maps using Plotly.
    Optimized for geographic data visualization and clustering analysis.
    """
    
    def __init__(self, title="Interactive Map", mapbox_style="open-street-map"):
        """
        Initialize the PlotlyMappingClient
        
        Args:
            title (str): Default title for maps
            mapbox_style (str): Map style ('open-street-map', 'carto-positron', 'carto-darkmatter', 'stamen-terrain', 'stamen-toner', 'stamen-watercolor')
        """
        self.title = title
        self.mapbox_style = mapbox_style
        self.color_palette = ['#FF0000', '#00FF00', '#0000FF', '#FFFF00', '#FF00FF', '#00FFFF', '#FFA500', '#800080', '#008000', '#FFC0CB']
        
    def create_scatter_map(self, data, lat_col, lon_col, color_col=None, size_col=None, 
                          hover_data=None, title=None, color_scale='Viridis', marker_size=8):
        """
        Create a scatter plot on geographic map
        
        Args:
            data: DataFrame with geographic data
            lat_col (str): Column name for latitude
            lon_col (str): Column name for longitude
            color_col (str): Column name for color mapping
            size_col (str): Column name for size mapping
            hover_data (list): Additional columns to show on hover
            title (str): Map title
            color_scale (str): Color scale for markers
            marker_size (int): Default marker size
            
        Returns:
            plotly.graph_objects.Figure: Interactive map figure
        """
        title = title or self.title
        
        if color_col and size_col:
            # Both color and size mapping
            fig = px.scatter_geo(data,
                               lat=lat_col,
                               lon=lon_col,
                               color=color_col,
                               size=size_col,
                               hover_data=hover_data,
                               title=title,
                               color_continuous_scale=color_scale)
        elif color_col:
            # Only color mapping
            fig = px.scatter_geo(data,
                               lat=lat_col,
                               lon=lon_col,
                               color=color_col,
                               hover_data=hover_data,
                               title=title,
                               color_continuous_scale=color_scale)
        elif size_col:
            # Only size mapping
            fig = px.scatter_geo(data,
                               lat=lat_col,
                               lon=lon_col,
                               size=size_col,
                               hover_data=hover_data,
                               title=title)
        else:
            # No color or size mapping
            fig = px.scatter_geo(data,
                               lat=lat_col,
                               lon=lon_col,
                               hover_data=hover_data,
                               title=title)
            fig.update_traces(marker=dict(size=marker_size))
        
        # Update layout for NYC region
        fig.update_geos(
            scope='usa',
            center=dict(lat=40.7128, lon=-74.0060),  # NYC center
            projection_scale=50,  # Zoom level
            showland=True,
            landcolor="rgb(243, 243, 243)",
            coastlinecolor="rgb(204, 204, 204)",
            showlakes=True,
            lakecolor="rgb(255, 255, 255)",
            showocean=True,
            oceancolor="rgb(230, 245, 255)"
        )
        
        return fig
    
    def create_cluster_map(self, data, lat_col, lon_col, cluster_col, cluster_centers=None,
                          hover_data=None, title=None, show_centers=True):
        """
        Create a map with clustered data points and cluster centers
        
        Args:
            data: DataFrame with clustered data
            lat_col (str): Column name for latitude
            lon_col (str): Column name for longitude
            cluster_col (str): Column name for cluster labels
            cluster_centers: DataFrame with cluster center coordinates
            hover_data (list): Additional columns to show on hover
            title (str): Map title
            show_centers (bool): Whether to show cluster centers
            
        Returns:
            plotly.graph_objects.Figure: Interactive cluster map
        """
        title = title or f"{self.title} - Cluster Analysis"
        
        fig = go.Figure()
        
        # Get unique clusters
        unique_clusters = data[cluster_col].unique()
        
        # Add data points for each cluster
        for i, cluster in enumerate(unique_clusters):
            cluster_data = data[data[cluster_col] == cluster]
            color = self.color_palette[i % len(self.color_palette)]
            
            # Create hover text
            hover_text = []
            if hover_data:
                for idx, row in cluster_data.iterrows():
                    text_parts = [f"Cluster: {cluster}"]
                    for col in hover_data:
                        if col in cluster_data.columns:
                            text_parts.append(f"{col}: {row[col]}")
                    hover_text.append("<br>".join(text_parts))
            else:
                hover_text = [f"Cluster: {cluster}"] * len(cluster_data)
            
            fig.add_trace(go.Scattergeo(
                lon=cluster_data[lon_col],
                lat=cluster_data[lat_col],
                text=hover_text,
                mode='markers',
                marker=dict(
                    size=8,
                    color=color,
                    opacity=0.7,
                    line=dict(width=1, color='white')
                ),
                name=f'Cluster {cluster}',
                hovertemplate='%{text}<extra></extra>'
            ))
        
        # Add cluster centers if provided
        if show_centers and cluster_centers is not None:
            fig.add_trace(go.Scattergeo(
                lon=cluster_centers[lon_col] if lon_col in cluster_centers.columns else cluster_centers['longitude'],
                lat=cluster_centers[lat_col] if lat_col in cluster_centers.columns else cluster_centers['latitude'],
                mode='markers',
                marker=dict(
                    size=20,
                    color='black',
                    symbol='star',
                    line=dict(width=2, color='white')
                ),
                name='Cluster Centers',
                hovertemplate='Cluster Center<br>Lat: %{lat}<br>Lon: %{lon}<extra></extra>'
            ))
        
        # Update layout
        fig.update_layout(
            title=title,
            geo=dict(
                scope='usa',
                center=dict(lat=40.7128, lon=-74.0060),  # NYC center
                projection_scale=50,
                showland=True,
                landcolor="rgb(243, 243, 243)",
                coastlinecolor="rgb(204, 204, 204)",
                showlakes=True,
                lakecolor="rgb(255, 255, 255)",
                showocean=True,
                oceancolor="rgb(230, 245, 255)"
            ),
            height=800
        )
        
        return fig
    
    def create_density_heatmap(self, data, lat_col, lon_col, z_col=None, title=None):
        """
        Create a density heatmap on geographic map
        
        Args:
            data: DataFrame with geographic data
            lat_col (str): Column name for latitude
            lon_col (str): Column name for longitude  
            z_col (str): Column name for density values (optional)
            title (str): Map title
            
        Returns:
            plotly.graph_objects.Figure: Interactive density heatmap
        """
        title = title or f"{self.title} - Density Heatmap"
        
        if z_col and z_col in data.columns:
            fig = px.density_mapbox(data, 
                                  lat=lat_col, 
                                  lon=lon_col, 
                                  z=z_col,
                                  radius=10,
                                  title=title,
                                  mapbox_style=self.mapbox_style)
        else:
            # Create density based on point concentration
            fig = px.density_mapbox(data, 
                                  lat=lat_col, 
                                  lon=lon_col,
                                  radius=10,
                                  title=title,
                                  mapbox_style=self.mapbox_style)
        
        fig.update_layout(
            mapbox=dict(
                center=dict(lat=40.7128, lon=-74.0060),  # NYC center
                zoom=10
            ),
            height=800
        )
        
        return fig
    
    def create_choropleth_map(self, data, locations_col, z_col, title=None, 
                             color_scale='Viridis', scope='usa'):
        """
        Create a choropleth map for regional data
        
        Args:
            data: DataFrame with regional data
            locations_col (str): Column with location codes (states, countries, etc.)
            z_col (str): Column with values to color by
            title (str): Map title
            color_scale (str): Color scale for regions
            scope (str): Geographic scope
            
        Returns:
            plotly.graph_objects.Figure: Interactive choropleth map
        """
        title = title or f"{self.title} - Regional Analysis"
        
        fig = px.choropleth(data,
                          locations=locations_col,
                          color=z_col,
                          title=title,
                          color_continuous_scale=color_scale,
                          scope=scope)
        
        return fig
    
    def save_map(self, fig, filename, format='html'):
        """
        Save map to file
        
        Args:
            fig: Plotly figure object
            filename (str): Output filename
            format (str): Output format ('html', 'png', 'pdf', 'svg')
        """
        if format == 'html':
            fig.write_html(filename)
        elif format == 'png':
            fig.write_image(filename)
        elif format == 'pdf':
            fig.write_image(filename)
        elif format == 'svg':
            fig.write_image(filename)
        else:
            raise ValueError(f"Unsupported format: {format}")
    
    def show_map(self, fig):
        """
        Display map in notebook
        
        Args:
            fig: Plotly figure object
        """
        fig.show()

# Initialize the PlotlyMappingClient
plotly_client = PlotlyMappingClient(title="NYC Motor Vehicle Collisions Analysis")
print("PlotlyMappingClient created successfully!")

PlotlyMappingClient created successfully!


In [9]:
# Import all required libraries after kernel restart
import polars as pl
import requests
import pandas as pd
from datetime import datetime, timedelta
from typing import Optional

print("All libraries imported successfully!")

All libraries imported successfully!


In [10]:
# Create NYC Open Data client for Polars
class NYCOpenDataPolarsClient:
    """
    NYC Open Data client optimized for Polars DataFrames
    """
    
    def __init__(self, app_token: Optional[str] = None):
        self.base_url = "https://data.cityofnewyork.us/resource"
        self.app_token = app_token
        self.session = requests.Session()
        
        if self.app_token:
            self.session.headers.update({'X-App-Token': self.app_token})
    
    def get_dataset_polars(self, dataset_id: str, **params) -> pl.DataFrame:
        """
        Fetch data and return as Polars DataFrame
        """
        url = f"{self.base_url}/{dataset_id}.json"
        
        try:
            response = self.session.get(url, params=params)
            response.raise_for_status()
            
            data = response.json()
            return pl.DataFrame(data)
            
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data: {e}")
            return pl.DataFrame()

# Create client instance
client = NYCOpenDataPolarsClient()
print("NYC Open Data Polars client created successfully!")

NYC Open Data Polars client created successfully!


In [11]:
# Fetch motor vehicle collision data
print("Fetching motor vehicle collision data...")

# Get recent collision data (last 30 days)
recent_date = (datetime.now() - timedelta(days=30)).strftime('%Y-%m-%d')

collisions_df = client.get_dataset_polars(
    "h9gi-nx95",
    **{
        "$limit": 5000,
        "$where": f"crash_date >= '{recent_date}'",
        "$order": "crash_date DESC"
    }
)

print(f"Fetched {len(collisions_df)} motor vehicle collision records")
print(f"Dataset shape: {collisions_df.shape}")
print(f"Columns: {collisions_df.columns[:10]}...")  # Show first 10 columns

Fetching motor vehicle collision data...
Fetched 5000 motor vehicle collision records
Dataset shape: (5000, 29)
Columns: ['crash_date', 'crash_time', 'borough', 'zip_code', 'latitude', 'longitude', 'location', 'cross_street_name', 'number_of_persons_injured', 'number_of_persons_killed']...


In [12]:
# Prepare location data for mapping
print("=== PREPARING LOCATION DATA FOR MAPPING ===")

# Extract location data with valid coordinates
location_data = (collisions_df
                .select([
                    "collision_id",
                    "latitude", 
                    "longitude",
                    "borough",
                    "on_street_name",
                    "cross_street_name",
                    "crash_date",
                    "crash_time",
                    "number_of_persons_injured",
                    "number_of_persons_killed",
                    "contributing_factor_vehicle_1"
                ])
                .filter(
                    pl.col("latitude").is_not_null() & 
                    pl.col("longitude").is_not_null() &
                    (pl.col("latitude") != "0") &
                    (pl.col("longitude") != "0")
                )
                .with_columns([
                    pl.col("latitude").cast(pl.Float64, strict=False),
                    pl.col("longitude").cast(pl.Float64, strict=False),
                    pl.col("number_of_persons_injured").cast(pl.Int32, strict=False).fill_null(0),
                    pl.col("number_of_persons_killed").cast(pl.Int32, strict=False).fill_null(0)
                ])
                .filter(
                    # Valid NYC coordinates roughly
                    (pl.col("latitude") > 40.4) & 
                    (pl.col("latitude") < 40.9) &
                    (pl.col("longitude") > -74.3) & 
                    (pl.col("longitude") < -73.7)
                ))

print(f"Total collisions with valid coordinates: {location_data.height}")
print(f"Original dataset size: {collisions_df.height}")

# Check what data we have
if not location_data.is_empty():
    print(f"\nLocation data sample:")
    print(location_data.head(3))
else:
    print("⚠️ No valid location data found")

=== PREPARING LOCATION DATA FOR MAPPING ===
Total collisions with valid coordinates: 4853
Original dataset size: 5000

Location data sample:
shape: (3, 11)
┌────────────┬───────────┬───────────┬─────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ collision_ ┆ latitude  ┆ longitude ┆ borough ┆ … ┆ crash_tim ┆ number_of ┆ number_of ┆ contribut │
│ id         ┆ ---       ┆ ---       ┆ ---     ┆   ┆ e         ┆ _persons_ ┆ _persons_ ┆ ing_facto │
│ ---        ┆ f64       ┆ f64       ┆ str     ┆   ┆ ---       ┆ injured   ┆ killed    ┆ r_vehicle │
│ str        ┆           ┆           ┆         ┆   ┆ str       ┆ ---       ┆ ---       ┆ _1        │
│            ┆           ┆           ┆         ┆   ┆           ┆ i32       ┆ i32       ┆ ---       │
│            ┆           ┆           ┆         ┆   ┆           ┆           ┆           ┆ str       │
╞════════════╪═══════════╪═══════════╪═════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 4840616    ┆ 40.757412 ┆ -73.83357

In [13]:
# Import clustering libraries and perform spatial clustering
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import numpy as np

print("=== PERFORMING SPATIAL CLUSTERING BY BOROUGH ===")

# Convert location_data to pandas for sklearn compatibility
location_df = location_data.to_pandas()

# Initialize results storage
borough_clusters = {}
cluster_centers_all = []

# Get unique boroughs (excluding nulls)
boroughs = location_df['borough'].dropna().unique()
print(f"Found boroughs: {boroughs}")

# Perform clustering for each borough
for borough in boroughs:
    borough_data = location_df[location_df['borough'] == borough].copy()
    
    if len(borough_data) < 10:  # Skip if too few data points
        print(f"Skipping {borough} - insufficient data ({len(borough_data)} points)")
        continue
    
    # Extract coordinates
    coordinates = borough_data[['latitude', 'longitude']].values
    
    # Standardize coordinates for clustering
    scaler = StandardScaler()
    coordinates_scaled = scaler.fit_transform(coordinates)
    
    # Determine optimal number of clusters (between 3-8)
    n_clusters = min(max(3, len(borough_data) // 200), 8)
    
    # Perform K-means clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(coordinates_scaled)
    
    # Add cluster labels to the data
    borough_data['cluster'] = cluster_labels
    
    # Get cluster centers in original coordinates
    cluster_centers = scaler.inverse_transform(kmeans.cluster_centers_)
    
    # Store results
    borough_clusters[borough] = {
        'data': borough_data,
        'centers': cluster_centers,
        'n_clusters': n_clusters
    }
    
    # Add to overall cluster centers with borough info
    for i, center in enumerate(cluster_centers):
        cluster_centers_all.append({
            'borough': borough,
            'cluster_id': i,
            'latitude': center[0],
            'longitude': center[1]
        })
    
    print(f"✅ {borough}: {len(borough_data)} collisions → {n_clusters} clusters")

# Convert cluster centers to DataFrame
cluster_centers_df = pl.DataFrame(cluster_centers_all)
print(f"\nTotal cluster centers: {len(cluster_centers_all)}")
print(f"Cluster centers shape: {cluster_centers_df.shape}")

# Combine all clustered data
all_clustered_data = []
for borough, info in borough_clusters.items():
    data = info['data'].copy()
    data['cluster_id'] = data['borough'] + '_' + data['cluster'].astype(str)
    all_clustered_data.append(data)

if all_clustered_data:
    combined_clustered_df = pl.from_pandas(pd.concat(all_clustered_data, ignore_index=True))
    print(f"Combined clustered data shape: {combined_clustered_df.shape}")
else:
    print("⚠️ No clustering data available")

=== PERFORMING SPATIAL CLUSTERING BY BOROUGH ===
Found boroughs: ['QUEENS' 'MANHATTAN' 'BROOKLYN' 'BRONX' 'STATEN ISLAND']
✅ QUEENS: 1034 collisions → 5 clusters
✅ MANHATTAN: 746 collisions → 3 clusters
✅ BROOKLYN: 1422 collisions → 7 clusters
✅ BRONX: 562 collisions → 3 clusters
✅ STATEN ISLAND: 171 collisions → 3 clusters

Total cluster centers: 21
Cluster centers shape: (21, 4)
Combined clustered data shape: (3935, 13)


In [14]:
# Create Plotly cluster visualization using our abstraction class
print("=== CREATING PLOTLY CLUSTER VISUALIZATIONS ===")

# 1. Create overall cluster map for all NYC boroughs
print("1. Creating overall NYC motor vehicle collision cluster map...")

cluster_map = plotly_client.create_cluster_map(
    data=combined_clustered_df.to_pandas(),
    lat_col='latitude',
    lon_col='longitude',
    cluster_col='cluster_id',
    cluster_centers=cluster_centers_df.to_pandas(),
    hover_data=['borough', 'crash_date', 'number_of_persons_injured', 'number_of_persons_killed', 'contributing_factor_vehicle_1'],
    title="NYC Motor Vehicle Collisions - Spatial Clustering Analysis",
    show_centers=True
)

# Display the map
plotly_client.show_map(cluster_map)
print("✅ Overall cluster map created and displayed!")

=== CREATING PLOTLY CLUSTER VISUALIZATIONS ===
1. Creating overall NYC motor vehicle collision cluster map...


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [1]:
# Quick setup after kernel restart for Plotly motor vehicle cluster visualization
import polars as pl
import pandas as pd
import requests
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
from typing import Optional
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import numpy as np

print("All libraries imported successfully!")

# Re-create the PlotlyMappingClient
class PlotlyMappingClient:
    """
    A comprehensive abstraction class for creating interactive maps using Plotly.
    Optimized for geographic data visualization and clustering analysis.
    """
    
    def __init__(self, title="Interactive Map", mapbox_style="open-street-map"):
        self.title = title
        self.mapbox_style = mapbox_style
        self.color_palette = ['#FF0000', '#00FF00', '#0000FF', '#FFFF00', '#FF00FF', '#00FFFF', '#FFA500', '#800080', '#008000', '#FFC0CB']
    
    def create_cluster_map(self, data, lat_col, lon_col, cluster_col, cluster_centers=None,
                          hover_data=None, title=None, show_centers=True):
        """Create a map with clustered data points and cluster centers"""
        title = title or f"{self.title} - Cluster Analysis"
        
        fig = go.Figure()
        
        # Get unique clusters
        unique_clusters = data[cluster_col].unique()
        
        # Add data points for each cluster
        for i, cluster in enumerate(unique_clusters):
            cluster_data = data[data[cluster_col] == cluster]
            color = self.color_palette[i % len(self.color_palette)]
            
            # Create hover text
            hover_text = []
            if hover_data:
                for idx, row in cluster_data.iterrows():
                    text_parts = [f"Cluster: {cluster}"]
                    for col in hover_data:
                        if col in cluster_data.columns:
                            text_parts.append(f"{col}: {row[col]}")
                    hover_text.append("<br>".join(text_parts))
            else:
                hover_text = [f"Cluster: {cluster}"] * len(cluster_data)
            
            fig.add_trace(go.Scattergeo(
                lon=cluster_data[lon_col],
                lat=cluster_data[lat_col],
                text=hover_text,
                mode='markers',
                marker=dict(
                    size=8,
                    color=color,
                    opacity=0.7,
                    line=dict(width=1, color='white')
                ),
                name=f'Cluster {cluster}',
                hovertemplate='%{text}<extra></extra>'
            ))
        
        # Add cluster centers if provided
        if show_centers and cluster_centers is not None:
            fig.add_trace(go.Scattergeo(
                lon=cluster_centers[lon_col] if lon_col in cluster_centers.columns else cluster_centers['longitude'],
                lat=cluster_centers[lat_col] if lat_col in cluster_centers.columns else cluster_centers['latitude'],
                mode='markers',
                marker=dict(
                    size=20,
                    color='black',
                    symbol='star',
                    line=dict(width=2, color='white')
                ),
                name='Cluster Centers',
                hovertemplate='Cluster Center<br>Lat: %{lat}<br>Lon: %{lon}<extra></extra>'
            ))
        
        # Update layout
        fig.update_layout(
            title=title,
            geo=dict(
                scope='usa',
                center=dict(lat=40.7128, lon=-74.0060),  # NYC center
                projection_scale=50,
                showland=True,
                landcolor="rgb(243, 243, 243)",
                coastlinecolor="rgb(204, 204, 204)",
                showlakes=True,
                lakecolor="rgb(255, 255, 255)",
                showocean=True,
                oceancolor="rgb(230, 245, 255)"
            ),
            height=800
        )
        
        return fig

# Initialize the client
plotly_client = PlotlyMappingClient(title="NYC Motor Vehicle Collisions Analysis")
print("PlotlyMappingClient initialized!")

All libraries imported successfully!
PlotlyMappingClient initialized!


In [2]:
# Quick data fetch and clustering for Plotly visualization
class NYCOpenDataPolarsClient:
    def __init__(self, app_token: Optional[str] = None):
        self.base_url = "https://data.cityofnewyork.us/resource"
        self.app_token = app_token
        self.session = requests.Session()
        if self.app_token:
            self.session.headers.update({'X-App-Token': self.app_token})
    
    def get_dataset_polars(self, dataset_id: str, **params) -> pl.DataFrame:
        url = f"{self.base_url}/{dataset_id}.json"
        try:
            response = self.session.get(url, params=params)
            response.raise_for_status()
            data = response.json()
            return pl.DataFrame(data)
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data: {e}")
            return pl.DataFrame()

# Fetch data
client = NYCOpenDataPolarsClient()
print("Fetching motor vehicle collision data...")

recent_date = (datetime.now() - timedelta(days=30)).strftime('%Y-%m-%d')
collisions_df = client.get_dataset_polars(
    "h9gi-nx95",
    **{
        "$limit": 5000,
        "$where": f"crash_date >= '{recent_date}'",
        "$order": "crash_date DESC"
    }
)

print(f"Fetched {len(collisions_df)} motor vehicle collision records")

# Prepare location data
location_data = (collisions_df
                .select([
                    "collision_id",
                    "latitude", 
                    "longitude",
                    "borough",
                    "on_street_name",
                    "crash_date",
                    "number_of_persons_injured",
                    "number_of_persons_killed",
                    "contributing_factor_vehicle_1"
                ])
                .filter(
                    pl.col("latitude").is_not_null() & 
                    pl.col("longitude").is_not_null() &
                    (pl.col("latitude") != "0") &
                    (pl.col("longitude") != "0")
                )
                .with_columns([
                    pl.col("latitude").cast(pl.Float64, strict=False),
                    pl.col("longitude").cast(pl.Float64, strict=False),
                    pl.col("number_of_persons_injured").cast(pl.Int32, strict=False).fill_null(0),
                    pl.col("number_of_persons_killed").cast(pl.Int32, strict=False).fill_null(0)
                ])
                .filter(
                    (pl.col("latitude") > 40.4) & 
                    (pl.col("latitude") < 40.9) &
                    (pl.col("longitude") > -74.3) & 
                    (pl.col("longitude") < -73.7)
                ))

print(f"Location data points: {location_data.height}")

# Quick clustering for demonstration
location_df = location_data.to_pandas()
borough_data = {}
cluster_centers_all = []

for borough in location_df['borough'].dropna().unique():
    borough_subset = location_df[location_df['borough'] == borough].copy()
    if len(borough_subset) >= 10:
        coords = borough_subset[['latitude', 'longitude']].values
        scaler = StandardScaler()
        coords_scaled = scaler.fit_transform(coords)
        
        n_clusters = min(max(3, len(borough_subset) // 200), 6)
        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
        cluster_labels = kmeans.fit_predict(coords_scaled)
        
        borough_subset['cluster'] = cluster_labels
        borough_subset['cluster_id'] = borough + '_' + borough_subset['cluster'].astype(str)
        borough_data[borough] = borough_subset
        
        # Get cluster centers
        centers = scaler.inverse_transform(kmeans.cluster_centers_)
        for i, center in enumerate(centers):
            cluster_centers_all.append({
                'borough': borough,
                'cluster_id': i,
                'latitude': center[0],
                'longitude': center[1]
            })

# Combine data
if borough_data:
    combined_data = pd.concat(list(borough_data.values()), ignore_index=True)
    cluster_centers_df = pd.DataFrame(cluster_centers_all)
    
    print(f"Combined clustered data: {len(combined_data)} points")
    print(f"Cluster centers: {len(cluster_centers_df)} centers")
    print(f"Boroughs processed: {list(borough_data.keys())}")
else:
    print("No clustering data available")

Fetching motor vehicle collision data...
Fetched 5000 motor vehicle collision records
Location data points: 4854
Combined clustered data: 3938 points
Cluster centers: 20 centers
Boroughs processed: ['QUEENS', 'BRONX', 'BROOKLYN', 'MANHATTAN', 'STATEN ISLAND']


In [3]:
# Create Plotly cluster visualization
print("=== CREATING PLOTLY MOTOR VEHICLE CLUSTER VISUALIZATION ===")

# Create the cluster map using our PlotlyMappingClient
cluster_map = plotly_client.create_cluster_map(
    data=combined_data,
    lat_col='latitude',
    lon_col='longitude',
    cluster_col='cluster_id',
    cluster_centers=cluster_centers_df,
    hover_data=['borough', 'crash_date', 'number_of_persons_injured', 'number_of_persons_killed', 'contributing_factor_vehicle_1'],
    title="NYC Motor Vehicle Collisions - Spatial Clustering Analysis (Plotly)",
    show_centers=True
)

# Display the interactive map
cluster_map.show()
print("✅ Interactive Plotly cluster map created and displayed!")

# Additional statistics
print(f"\n=== CLUSTER ANALYSIS SUMMARY ===")
print(f"Total collision points: {len(combined_data):,}")
print(f"Total cluster centers: {len(cluster_centers_df)}")
print(f"Boroughs analyzed: {', '.join(combined_data['borough'].unique())}")

# Borough breakdown
print(f"\n=== BOROUGH BREAKDOWN ===")
borough_stats = combined_data['borough'].value_counts()
for borough, count in borough_stats.items():
    clusters = len(combined_data[combined_data['borough'] == borough]['cluster'].unique())
    print(f"• {borough}: {count:,} collisions in {clusters} clusters")

# Injury statistics by cluster
print(f"\n=== INJURY STATISTICS ===")
injury_stats = combined_data.groupby('borough').agg({
    'number_of_persons_injured': 'sum',
    'number_of_persons_killed': 'sum'
}).round(2)
print(injury_stats)

=== CREATING PLOTLY MOTOR VEHICLE CLUSTER VISUALIZATION ===


✅ Interactive Plotly cluster map created and displayed!

=== CLUSTER ANALYSIS SUMMARY ===
Total collision points: 3,938
Total cluster centers: 20
Boroughs analyzed: QUEENS, BRONX, BROOKLYN, MANHATTAN, STATEN ISLAND

=== BOROUGH BREAKDOWN ===
• BROOKLYN: 1,428 collisions in 6 clusters
• QUEENS: 1,036 collisions in 5 clusters
• MANHATTAN: 746 collisions in 3 clusters
• BRONX: 557 collisions in 3 clusters
• STATEN ISLAND: 171 collisions in 3 clusters

=== INJURY STATISTICS ===
               number_of_persons_injured  number_of_persons_killed
borough                                                           
BRONX                                361                         2
BROOKLYN                             906                         4
MANHATTAN                            410                         1
QUEENS                               614                         2
STATEN ISLAND                         88                         0


In [4]:
# Create additional Plotly visualizations using our abstraction class

print("=== ADDITIONAL PLOTLY MAPPING VISUALIZATIONS ===")

# 1. Simple scatter map colored by borough
print("1. Creating borough-based scatter map...")

# Add scatter_geo method to our client
def create_scatter_map_geo(self, data, lat_col, lon_col, color_col=None, size_col=None, 
                      hover_data=None, title=None, color_scale='Viridis'):
    """Create a scatter plot on geographic map using px.scatter_geo"""
    title = title or self.title
    
    if color_col and size_col:
        fig = px.scatter_geo(data,
                           lat=lat_col,
                           lon=lon_col,
                           color=color_col,
                           size=size_col,
                           hover_data=hover_data,
                           title=title,
                           color_continuous_scale=color_scale)
    elif color_col:
        fig = px.scatter_geo(data,
                           lat=lat_col,
                           lon=lon_col,
                           color=color_col,
                           hover_data=hover_data,
                           title=title)
    else:
        fig = px.scatter_geo(data,
                           lat=lat_col,
                           lon=lon_col,
                           hover_data=hover_data,
                           title=title)
    
    # Update layout for NYC region
    fig.update_geos(
        scope='usa',
        center=dict(lat=40.7128, lon=-74.0060),
        projection_scale=50,
        showland=True,
        landcolor="rgb(243, 243, 243)",
        coastlinecolor="rgb(204, 204, 204)"
    )
    
    return fig

# Add method to class
PlotlyMappingClient.create_scatter_map_geo = create_scatter_map_geo

# Create borough-colored scatter map
borough_map = plotly_client.create_scatter_map_geo(
    data=combined_data,
    lat_col='latitude',
    lon_col='longitude',
    color_col='borough',
    hover_data=['crash_date', 'number_of_persons_injured', 'contributing_factor_vehicle_1'],
    title="NYC Motor Vehicle Collisions by Borough"
)

borough_map.show()
print("✅ Borough-based scatter map created!")

# 2. Injury severity map
print("\n2. Creating injury severity visualization...")

# Create severity categories
combined_data['severity'] = pd.cut(
    combined_data['number_of_persons_injured'] + combined_data['number_of_persons_killed'], 
    bins=[-1, 0, 1, 2, float('inf')], 
    labels=['No Injury', 'Minor', 'Moderate', 'Severe']
)

severity_map = plotly_client.create_scatter_map_geo(
    data=combined_data,
    lat_col='latitude',
    lon_col='longitude',
    color_col='severity',
    hover_data=['borough', 'crash_date', 'number_of_persons_injured', 'number_of_persons_killed'],
    title="NYC Motor Vehicle Collisions by Injury Severity"
)

severity_map.show()
print("✅ Injury severity map created!")

print(f"\n=== SEVERITY BREAKDOWN ===")
severity_counts = combined_data['severity'].value_counts()
for severity, count in severity_counts.items():
    percentage = (count / len(combined_data)) * 100
    print(f"• {severity}: {count:,} collisions ({percentage:.1f}%)")

=== ADDITIONAL PLOTLY MAPPING VISUALIZATIONS ===
1. Creating borough-based scatter map...


✅ Borough-based scatter map created!

2. Creating injury severity visualization...


✅ Injury severity map created!

=== SEVERITY BREAKDOWN ===
• No Injury: 2,101 collisions (53.4%)
• Minor: 1,476 collisions (37.5%)
• Moderate: 238 collisions (6.0%)
• Severe: 123 collisions (3.1%)


In [5]:
# Final summary and comparison with Folium
print("=== PLOTLY MAPPING IMPLEMENTATION COMPLETE ===")
print()
print("🎯 SUCCESSFULLY IMPLEMENTED:")
print("✅ PlotlyMappingClient abstraction class")
print("✅ Motor vehicle collision cluster visualization") 
print("✅ Borough-based color mapping")
print("✅ Injury severity analysis")
print("✅ Interactive hover data and legends")
print("✅ Cluster centers with star markers")
print()

print("📊 PLOTLY vs FOLIUM COMPARISON:")
print()
print("PLOTLY ADVANTAGES:")
print("• Native Jupyter integration with nbformat")
print("• Better performance with large datasets")
print("• More sophisticated clustering visualization")
print("• Advanced interactivity (zoom, pan, legend controls)")
print("• Professional data visualization styling")
print("• Seamless integration with pandas/polars")
print("• Built-in geographic projections and scopes")
print()

print("FOLIUM ADVANTAGES:")
print("• Direct OpenStreetMap integration")
print("• Simpler HTML export")
print("• Lighter weight for basic mapping")
print("• Better for simple point-and-click maps")
print()

print("🗺️ VISUALIZATION SUMMARY:")
print(f"• Total collision points visualized: {len(combined_data):,}")
print(f"• Boroughs analyzed: {len(combined_data['borough'].unique())}")
print(f"• Cluster centers identified: {len(cluster_centers_df)}")
print(f"• Date range: Last 30 days")
print(f"• Geographic scope: NYC metropolitan area")
print()

print("🔧 PLOTLY ABSTRACTION CLASS FEATURES:")
print("• create_cluster_map() - Spatial clustering with centers")
print("• create_scatter_map_geo() - Geographic scatter plots")
print("• Configurable color palettes and styling")
print("• Hover data customization")
print("• NYC-optimized geographic projections")
print("• Flexible data input (pandas/polars)")
print()

print("🚀 IMPLEMENTATION SUCCESS!")
print("The PlotlyMappingClient provides a comprehensive abstraction")
print("for creating professional interactive maps with clustering analysis.")
print("Perfect for motor vehicle collision analysis and similar geospatial datasets.")

=== PLOTLY MAPPING IMPLEMENTATION COMPLETE ===

🎯 SUCCESSFULLY IMPLEMENTED:
✅ PlotlyMappingClient abstraction class
✅ Motor vehicle collision cluster visualization
✅ Borough-based color mapping
✅ Injury severity analysis
✅ Interactive hover data and legends
✅ Cluster centers with star markers

📊 PLOTLY vs FOLIUM COMPARISON:

PLOTLY ADVANTAGES:
• Native Jupyter integration with nbformat
• Better performance with large datasets
• More sophisticated clustering visualization
• Advanced interactivity (zoom, pan, legend controls)
• Professional data visualization styling
• Seamless integration with pandas/polars
• Built-in geographic projections and scopes

FOLIUM ADVANTAGES:
• Direct OpenStreetMap integration
• Simpler HTML export
• Lighter weight for basic mapping
• Better for simple point-and-click maps

🗺️ VISUALIZATION SUMMARY:
• Total collision points visualized: 3,938
• Boroughs analyzed: 5
• Cluster centers identified: 20
• Date range: Last 30 days
• Geographic scope: NYC metropolita